# Hello, FABRIC: Create your First FABRIC Eperiment

This Jupyter notebook will walk you through creating your first FABRIC experiment. The "Hello, FABRIC" experiment tests the configuration of your environment to ensure you can create and access resources on FABRIC.  Specifically, the experiment deploys a slice of resources with a single virtual machine and confirms you can log into the machine. 

## Step 1:  Configure the Environment

Set the environment variables that will be used by this notebook. If you are using the FABRIC JupyterHub many of the environment varialbe will be automatically configured for you.  You will only need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  If you do not have a bastion username and private key, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

If you are using the FABRIC API outside of the JupyterHub you will need to configure all of the environment variables. Defaults below will be correct in many situations but you will need to confirm your configuration.  If you have questions about this configuration, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).

In [1]:
import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
os.environ['FABRIC_CREDMGR_HOST']='beta-2.fabric-testbed.net'
os.environ['FABRIC_ORCHESTRATOR_HOST']='beta-7.fabric-testbed.net'
os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_config/fabric_token.json'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']='pruth_0031379841'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/.ssh/id_ecdsa_fabric_bastion'

# Set the keypair FABRIC will install in your slice. 
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/work/fabric_config/slice-private-key'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/work/fabric_config/slice-public-key'


os.environ['FABRIC_PROJECT_ID']='b9847fa1-13ef-49f9-9e07-ae6ad06cda3f'
# If your slice private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase. Press enter for no passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

## Step 2: Import the FABLlib Library

In [2]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib

## Step 3 (Optional): Query for Available Tesbed Resources and Settings

This optional command queries the FABRIC services to find the available resources. It may be useful for finding a site with available capacity.

In [3]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

Name      CPUs  Cores    RAM (G)    Disk (G)     Basic (100 Gbps NIC)    ConnectX-6 (100 Gbps x2 NIC)    ConnectX-5 (25 Gbps x2 NIC)    P4510 (NVMe 1TB)    Tesla T4 (GPU)    RTX6000 (GPU)
------  ------  -------  ---------  -----------  ----------------------  ------------------------------  -----------------------------  ------------------  ----------------  ---------------
LBNL         6  192/192  1536/1536  60600/60600  381/381                 2/2                             2/2                            10/10               4/4               2/2
RENC         6  190/192  1530/1536  60590/60600  381/381                 2/2                             2/2                            10/10               4/4               2/2
UKY          6  190/192  1530/1536  60590/60600  381/381                 2/2                             2/2                            10/10               4/4               2/2


## Step 4: Create the Experiment Slice

The following creates a single node with basic compute capabilities. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated.   

By default, the submit function will block until the node is ready and will display the progress of your slice being built.


<img src="./figs/SingleNode.png" width="20%"><br>



In [4]:
try:
    #Create a slice
    slice = fablib.new_slice(name="MySlice4")

    # Add a node
    node = slice.add_node(name="Node1", site='RENC')

    #Submit the Request
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")


-----------  ------------------------------------
Slice Name   MySlice4
Slice ID     5b3dde6c-0f60-4814-a489-038a944aa9c0
Slice State  StableOK
Lease End    2022-05-18 20:52:36 +0000
-----------  ------------------------------------

Retry: 16, Time: 175 sec

ID                                    Name    Site    Host                          Cores    RAM    Disk  Image            Management IP    State    Error
------------------------------------  ------  ------  --------------------------  -------  -----  ------  ---------------  ---------------  -------  -------
841e85ed-6343-4746-a4a0-80efefd1c101  Node1   RENC    renc-w1.fabric-testbed.net        2      8      10  default_rocky_8  152.54.15.40     Active

Time to stable 175 seconds
Running wait_ssh ... Time to ssh 176 seconds
Running post_boot_config ... Time to post boot config 178 seconds


## Step 5: Observe the Slice's Attributes

### Print the slice 

In [5]:
try:
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

-----------  ------------------------------------
Slice Name   MySlice4
Slice ID     5b3dde6c-0f60-4814-a489-038a944aa9c0
Slice State  StableOK
Lease End    2022-05-18 20:52:36 +0000
-----------  ------------------------------------


### Print the node

Each node in the slice has a set of get functions that return the node's attributes.

In [6]:
try:
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

-----------------  ----------------------------------------------------------------------------------------------------------------------------
ID                 841e85ed-6343-4746-a4a0-80efefd1c101
Name               Node1
Cores              2
RAM                8
Disk               10
Image              default_rocky_8
Image Type         qcow2
Host               renc-w1.fabric-testbed.net
Site               RENC
Management IP      152.54.15.40
Reservation State  Active
Error Message
SSH Command        ssh -i /Users/pruth/work/fabric_config/slice-private-key -J pruth_0031379841@bastion-1.fabric-testbed.net rocky@152.54.15.40
-----------------  ----------------------------------------------------------------------------------------------------------------------------


## Step 6: Run the Experiemnt

Most experiments will require automated configuration and execution. You can use the fablib library to execute arbitrary commands on your node. 

The following code demonstrates how to use fablib to execute a "Hello, FABRIC" bash script. The library uses the bastion and VM keys defined at the top of this notebook to jump through the bastion host and execute the script.

In [7]:
try:
    for node in slice.get_nodes():
        stdout, stderr = node.execute('echo Hello, FABRIC from node `hostname -s`')
        print(stdout)
except Exception as e:
    print(f"Exception: {e}")

Hello, FABRIC from node 841e85ed-6343-4746-a4a0-80efefd1c101-node1



## Step 7: Delete the Slice

Please delete your slice when you are done with your experiment.

In [8]:
try:
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")